In [229]:
# copy(select companies.name, emails.guid, emails.subject, emails.full_text from emails join companies on emails.company_id = companies.id join companies_tags on companies.id = companies_tags.company_id where companies_tags.tag_id = 1100 and emails.entity_state = 'original' order by emails.created_at desc limit 500) to '/tmp/emails.csv' with CSV HEADER;
# data: http://share.mailcharts.com/0L193K121x1l
# https://docs.google.com/spreadsheets/d/1XAYfOeRTxcebdt0ALm0ExsxhK0bodrx-bfEKA9ct7GE/edit#gid=0

import pandas as pd
import numpy as np
import math
import re
import time

In [231]:
def combine_categories(row):
    result = row.category_1.lower()
    
    if row.category_2:
        result = result + " > " + row.category_2.lower()
    if row.category_3:
        result = result + " > " + row.category_3.lower()
    if row.category_4:
        result = result + " > " + row.category_4.lower()
    if row.category_5:
        result = result + " > " + row.category_5.lower()
    if row.category_6:
        result = result + " > " + row.category_6.lower()
    if row.category_7:
        result = result + " > " + row.category_7.lower()

    return result

def get_last_category_val(row):    
    if row.category_7:
        return row.category_7.lower()
    if row.category_6:
        return row.category_6.lower()
    if row.category_5:
        return row.category_5.lower()
    if row.category_4:
        return row.category_4.lower()
    if row.category_3:
        return row.category_3.lower()
    if row.category_2:
        return row.category_2.lower()
    if row.category_1:
        return row.category_1.lower()

In [232]:
# https://www.google.com/basepages/producttype/taxonomy.en-US.txt
cat = pd.read_csv("./data/taxonomy-carl.csv", encoding="ISO-8859-1", dtype=str)
cat.fillna(False, inplace=True)
cat["combined"] = cat.apply(lambda x: combine_categories(x), axis=1)
cat["lookup_value"] = cat.apply(lambda x: get_last_category_val(x), axis=1)
print(cat.shape)
cat.head()

(103, 9)


,category_1,category_2,category_3,category_4,category_5,category_6,category_7,combined,lookup_value
0,clothing,False,False,False,False,False,False,clothing,clothing
1,clothing,outerwear,False,False,False,False,False,clothing > outerwear,outerwear
2,clothing,outerwear,blazer,False,False,False,False,clothing > outerwear > blazer,blazer
3,clothing,outerwear,coat,False,False,False,False,clothing > outerwear > coat,coat
4,clothing,outerwear,rain jacket,False,False,False,False,clothing > outerwear > rain jacket,rain jacket


In [243]:
df = pd.read_csv("./data/emails-tag-1100.csv", encoding="ISO-8859-1", dtype=str)
df["all_text"] = df["subject"] + " " + df["full_text"]
df["all_text"] = df["all_text"].apply(lambda x: str(x).lower())
df = df.iloc[:50]
print(df.shape)
df.head()

(50, 5)


,name,guid,subject,full_text,all_text
0,Secret Sales UK,8caef803-009b-7751-312c-52f8d0181565,Beat The January Blues with up to 60% off Seve...,"Plus: John & Yoko Shearling, Vintage Louis Vui...",beat the january blues with up to 60% off seve...
1,Fashion Eyewear,3b9666be-1ddc-952f-329e-6e832911892e,January Sale Final Weekend | Up to 30% OFF!,Get 20% OFF using code FRESH20 No Images? Clic...,january sale final weekend | up to 30% off! ge...
2,Betabrand,0e24b7c3-1971-bd90-e271-2050cd50159c,Sasquatch Conquers All!,"[A new shirt celebrating nature's most brutal,...",sasquatch conquers all! [a new shirt celebrati...
3,Free People,6ec4361a-cd9b-dd16-84cc-7257447f7071,WANTEDð: That Femme Outfit,Whatâs New at Free People Sweet-meets-sensua...,wantedð: that femme outfit whatâs new at ...
4,BOOM by Cindy Joseph,bee78052-5639-435c-87c4-38ca892ff505,âI felt liberated. ____was my first step to ...,BOOM! by Cindy Joseph: It's about women. It's ...,âi felt liberated. ____was my first step to ...


In [234]:
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')

all_text = df.all_text.apply(lambda x: x.lower())

start_time = time.time()
categories = []

for text in all_text:
    category = set()
    for i, row in cat.iterrows():
        category_to_find = row["lookup_value"]
        if contains_word(text, category_to_find):
            category.add(i)
    categories.append(list(category))

df["category_ids"] = categories

In [235]:
# all_text = df.all_text.apply(lambda x: x.lower())

# start_time = time.time()
# categories = []

# for text in all_text:
#     category = set()
#     text = set(re.split(r"[\>|\&| ]+", text))

#     for i, row in cat.iterrows():
#         words = row["combined_set"]
        
#         if len(words.intersection(text))>0: 
#             category.add(i)

#     categories.append(list(category))

# print("--- %s seconds ---" % (time.time() - start_time))
# # --- 3.6054348945617676 seconds ---
# df["category_ids"] = categories

In [236]:
# def print_categories(list_of_ids):
#     for i in list_of_ids:
#         category = cat.iloc[i, :-1]
#         print(" > ".join([i for i in category if i != False]))
        
# print_categories(list(category))

In [237]:
def convert_categories_index_to_text(categories):
    result = []
    if len(categories) == 0:
        return []
    for i in categories:
        result.append(cat.iloc[i]["combined"])
        result.append("*****")
    return result

df["categories"] = df["category_ids"].apply(lambda x: convert_categories_index_to_text(x))


In [238]:
df

,name,guid,subject,full_text,all_text,category_ids,categories
0,Secret Sales UK,8caef803-009b-7751-312c-52f8d0181565,Beat The January Blues with up to 60% off Seve...,"Plus: John & Yoko Shearling, Vintage Louis Vui...",beat the january blues with up to 60% off seve...,"[43, 37, 55]","[clothing > dress, *****, clothing > underwear..."
1,Fashion Eyewear,3b9666be-1ddc-952f-329e-6e832911892e,January Sale Final Weekend | Up to 30% OFF!,Get 20% OFF using code FRESH20 No Images? Clic...,january sale final weekend | up to 30% off! ge...,[80],"[accessory > sunglasses, *****]"
2,Betabrand,0e24b7c3-1971-bd90-e271-2050cd50159c,Sasquatch Conquers All!,"[A new shirt celebrating nature's most brutal,...",sasquatch conquers all! [a new shirt celebrati...,"[8, 9, 67, 76, 23]","[clothing > outerwear > jacket, *****, clothin..."
3,Free People,6ec4361a-cd9b-dd16-84cc-7257447f7071,WANTEDð: That Femme Outfit,Whatâs New at Free People Sweet-meets-sensua...,wantedð: that femme outfit whatâs new at ...,[],[]
4,BOOM by Cindy Joseph,bee78052-5639-435c-87c4-38ca892ff505,âI felt liberated. ____was my first step to ...,BOOM! by Cindy Joseph: It's about women. It's ...,âi felt liberated. ____was my first step to ...,[76],"[accessory > watch, *****]"
5,Anthropologie,aab73866-8b24-58e1-7f5b-c50ea44476c5,How to wear indigo.,We're loving the blues. ----------------------...,how to wear indigo. we're loving the blues. --...,[11],"[clothing > pants > denim, *****]"
6,Bluefly,0e448959-a6f9-a7cb-cd6e-7e565b9313ad,(Hint: Diamonds On Sale) Up To 75% Off Fine Je...,"Plus, Extra 20% Off Dresses | View On Web Brow...",(hint: diamonds on sale) up to 75% off fine je...,"[89, 43]","[accessory > jewelry, *****, clothing > dress,..."
7,Endource Limited,0f9420b2-4d5a-2233-1009-845f29eb3b78,A shortcut to Parisian chic,| Bring the fitting room to your home | 663399...,a shortcut to parisian chic | bring the fittin...,[],[]
8,LE CHÃTEAU,90c770fa-88fc-cecc-2c38-774f9e1166fc,"Flash Sale: 1,500 styles at $50 or less!",Seriously. Low. Prices. View in browser Free S...,"flash sale: 1,500 styles at $50 or less! serio...",[],[]
9,Alex Monroe,407155a8-b8e2-b1d0-c346-32736b50603a,Final Weekend of our Archive Sale! | Alex talk...,It's the final days of our Archive Sale and a ...,final weekend of our archive sale! | alex talk...,"[76, 92]","[accessory > watch, *****, accessory > jewelry..."


In [241]:
# QA
## URL: https://www.mailcharts.com/emails/_GUID_
## Wrong (from file #1: 'emails.csv'): 11, 14, 15, 24, 25

position = 2

print(df.iloc[position]["all_text"])
print("========")
print(df.iloc[position]["categories"])

sasquatch conquers all! [a new shirt celebrating nature's most brutal, hirsute foe.] sasquatch: apex predator watch him crush god's creatures with ravenous bloodlust! listen as they beg for mercy! alas, sasquatch has none! sasquatch: apex predator (hawaiian shirt) woe be to any man or beast who dares challenge mighty sasquatch! if you thought he was but a shy, woodland herbivore, you'd be wrong. catastrophically, jaw-rippingly wrong. sasquatch: apex predator (hawaiian shirt) with strength, speed, and bravery not seen since the cretaceous period, this brute supreme dispenses horrifying justice upon all who cross his path! sasquatch: apex predator (hawaiian shirt) he gorges upon great whites, ko's kodiak bears, and tears poor tigers asunder! (now you know why so many members of the lewis and clark expedition lost arms and faces.) sasquatch: apex predator (hawaiian shirt) reserve your shirt now, and soon you can show the world that sasquatch means business. sasquatch: apex predator (hawai

In [110]:
# next steps
# how do we handle singular v.s. plural? (both in lookup table + when classifing — e.g. 12 (rompers vs. romper))
# how do we handle synonyms (e.g. swimwear & bathing suit)
# should we remove stop words?
# how can we weight things (e.g. subject weights more than body)
# how can we get a score (and remove anything below X score)
# how to handle ambiguous terms (watch: clock) v.s. (watch: verb)

# using google vision to understand the email better (image recognition, OCR, etc)
### use this info to add new categories
# explore using different weights (would this make sense for us?)

In [111]:
# We tried using tfidf to look values up instead of our naive lookup approach
# This did not yield better results


# from sklearn.feature_extraction.text import TfidfVectorizer

# cat_tfidf = TfidfVectorizer()
# cat_tfidf = cat_tfidf.fit(cat.combined)
# cat_tfid_t = cat_tfidf.transform(cat.combined)

# test = df.iloc[2].subject
# print(test)

# test_vec = cat_tfidf.transform([test])

# from sklearn.metrics.pairwise import cosine_similarity
# cos = cosine_similarity(test_vec, cat_tfid_t)
# cat.iloc[cos.argmax()].combined

In [156]:
import string
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [148]:
# Get stopwords_set and punctuation set.
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)

In [151]:
all_words_in_subjects = []

for i, row in df.iterrows():
    word_tokens = word_tokenize(row.subject)
    filtered_sentence = [w.lower() for w in word_tokens if not w in stopwords and not w in punctuation and w.isalpha()]
    all_words_in_subjects.extend(filtered_sentence)
    
    word_tokens = word_tokenize(row.full_text)
    filtered_sentence = [w.lower() for w in word_tokens if not w in stopwords and not w in punctuation and w.isalpha()]
    all_words_in_subjects.extend(filtered_sentence)

In [155]:
# basic word count
from collections import Counter
counts = Counter(all_words_in_subjects)
counts.most_common()

[('the', 149),
 ('share', 137),
 ('email', 122),
 ('free', 99),
 ('new', 95),
 ('shipping', 94),
 ('i', 72),
 ('valid', 66),
 ('shop', 63),
 ('us', 62),
 ('more', 61),
 ('you', 57),
 ('unsubscribe', 54),
 ('stores', 46),
 ('online', 46),
 ('now', 45),
 ('click', 45),
 ('see', 44),
 ('view', 44),
 ('vox', 43),
 ('use', 43),
 ('this', 40),
 ('policy', 40),
 ('get', 40),
 ('offer', 40),
 ('may', 40),
 ('please', 40),
 ('to', 39),
 ('read', 39),
 ('privacy', 39),
 ('one', 38),
 ('your', 38),
 ('not', 35),
 ('address', 35),
 ('facebook', 35),
 ('news', 34),
 ('code', 33),
 ('sales', 33),
 ('place', 33),
 ('would', 33),
 ('orders', 32),
 ('women', 32),
 ('gift', 31),
 ('could', 31),
 ('price', 31),
 ('card', 29),
 ('change', 29),
 ('find', 29),
 ('in', 29),
 ('day', 29),
 ('like', 28),
 ('if', 27),
 ('total', 27),
 ('twitter', 26),
 ('make', 26),
 ('de', 26),
 ('no', 26),
 ('purchases', 25),
 ('terms', 25),
 ('marketing', 25),
 ('lowest', 24),
 ('all', 24),
 ('march', 24),
 ('know', 24),
 ('

In [161]:
syns = wordnet.synsets('stack')

In [177]:
for s in syns:
    for l in s.lemmas():
        print(str(l.name()) + " " + str(l.count()))
        break

stack 2
batch 0
push-down_list 0
smokestack 0
push-down_storage 0
stack 2
stack 0
stack 0


In [189]:
i = 0
syns[i].lemmas()[i].name()

'stack'

In [53]:
## Next steps
# Get scraper working: grab top 20 words on 5 websites
# Get word freq and see if anything stands out
## Alternatives
# Team manually builds the word list
# 